In [1]:
# t_loss_bkwd vs t_loss_bkwd2 on train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
#from model_triton import *
from loss_and_optimizer_triton import t_loss, t_loss_bkwd, t_loss_bkwd2
BS, H, N, D = 2, 2, 5, 4 #1, 1, 3, 4 #2, 2, 5, 4
vocab_size = 4
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cuda").to(torch.int32)
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cuda")

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
# print(mask)

res2 = t_loss_bkwd(layers_params, y, mask, None, True, p_gen_aux)
#print(res2[1])
print_res_shapes(res2[0]) 

print(f'----XXX----')

res3 = t_loss_bkwd2(layers_params, y, mask, None, True, p_gen_aux)
#print(res3)
print_res_shapes(res3[0]) 


5
- 2
-- torch.Size([4, 4]) tensor([-0.217906, -0.555803,  0.241282], device='cuda:0')
-- torch.Size([4]) tensor([ 0.065787,  0.034790, -0.352987], device='cuda:0')
- 1
-- torch.Size([5, 4]) tensor([0., 0., 0.], device='cuda:0')
- 10
-- torch.Size([4]) tensor([-2.869071e-05,  3.290028e-06, -2.191706e-05], device='cuda:0')
-- torch.Size([4]) tensor([ 2.961929e-05, -1.457916e-05, -1.047885e-05], device='cuda:0')
-- torch.Size([2, 3, 2, 4]) tensor([-7.073849e-05, -5.367380e-04,  1.783088e-04], device='cuda:0')
-- torch.Size([4, 4]) tensor([ 0.000152, -0.003612,  0.003850], device='cuda:0')
-- torch.Size([4]) tensor([ 8.589582e-05, -1.060286e-05, -2.828828e-05], device='cuda:0')
-- torch.Size([4]) tensor([-3.337320e-05, -8.145726e-07, -1.067495e-05], device='cuda:0')
-- torch.Size([16, 4]) tensor([-9.771282e-04,  5.700602e-05,  2.485688e-03], device='cuda:0')
-- torch.Size([16]) tensor([1.256306e-04, 1.362190e-05, 6.952654e-04], device='cuda:0')
-- torch.Size([4, 16]) tensor([ 0.000229,  0

In [2]:
-1.325236e-05 * -0.500069 + -1.871210e-05 * -0.499931

1.598185327794e-05

In [3]:
-1.325236e-05 * -0.499931  + -1.871210e-05 * -0.500069

1.598260672206e-05

In [4]:
aa = torch.zeros((3,6), dtype=torch.float32)
y_labels_1d = torch.tensor([[0, 4, 2]], dtype=torch.int64).t()
print(y_labels_1d.shape, y_labels_1d)
dloss_dx = torch.tensor([[0.2, 0.1, 0.1]], dtype=torch.float32).t()
print(dloss_dx.shape, dloss_dx)
aa.scatter_(1,y_labels_1d,dloss_dx)
aa

torch.Size([3, 1]) tensor([[0],
        [4],
        [2]])
torch.Size([3, 1]) tensor([[0.200000],
        [0.100000],
        [0.100000]])


tensor([[0.200000, 0.000000, 0.000000, 0.000000, 0.000000, 0.000000],
        [0.000000, 0.000000, 0.000000, 0.000000, 0.100000, 0.000000],
        [0.000000, 0.000000, 0.100000, 0.000000, 0.000000, 0.000000]])

In [5]:
#B jac_log_softmax.shape torch.Size([2, 2, 3, 3, 2, 2, 3, 3]) q.shape torch.Size([2, 2, 3, 2])
#B jac_k torch.Size([2, 2, 3, 3, 2, 2, 3, 2])
#B q[None, None, None, None, ...].shape torch.Size([1, 1, 1, 1, 2, 2, 3, 2])
jac_log_softmax = torch.randn(([2, 2, 3, 3, 2, 2, 3, 3]))
q = torch.randn(([1, 1, 1, 1, 2, 2, 3, 2]))
torch.matmul(jac_log_softmax, q)

tensor([[[[[[[[-1.333067e-01, -1.521276e+00],
              [ 5.075805e-01,  8.676277e-01],
              [ 2.688751e-02,  1.454227e+00]],

             [[ 1.401155e+00,  2.209958e+00],
              [ 6.169189e-01,  9.020051e-01],
              [-1.047605e+00, -3.552344e+00]]],


            [[[-4.061303e-01, -3.951000e-01],
              [ 5.932398e-01,  8.480312e-01],
              [ 7.753129e-01, -6.771272e-01]],

             [[ 4.661110e-02, -7.494920e-01],
              [ 1.016872e+00, -4.847762e-01],
              [-3.924007e-01, -4.475892e-01]]]],



           [[[[ 5.382323e-01,  1.107040e+00],
              [ 9.273115e-01,  1.595398e-01],
              [ 3.480131e-01, -2.458797e-01]],

             [[ 1.675244e+00, -5.302330e-01],
              [-2.682544e-01, -1.097696e+00],
              [ 1.730470e+00, -7.147262e-02]]],


            [[[-5.400551e-01,  3.159516e-02],
              [-1.879472e+00,  4.032625e-01],
              [ 6.862844e-01,  1.294222e-01]],

            

In [6]:
%env CUDA_VISIBLE_DEVICES=""

# t_avg_cross_entropy_loss_bkwd vs t_avg_cross_entropy_loss_bkwd2 on train=True
import torch
torch.set_printoptions(precision=6)
from torch.func import jacrev
from model_torch_func import init_transformer_gpt2
from model_triton import t_gpt2_forward
from loss_and_optimizer_triton import t_avg_cross_entropy_loss_bkwd, t_avg_cross_entropy_loss_bkwd2
BS, H, N, D = 2, 2, 5, 4
vocab_size = 6
layers = 2
p_gen_aux = [42] + [43,44,45] * layers
layers_params = init_transformer_gpt2(vocab_size, D, layers, H, 4*D, N)
y= torch.randint(vocab_size, (BS, N+1), device="cpu")
mask = torch.ones((BS, N, N), dtype=torch.bool, device="cpu")

y_in = y[:, :-1]
y_out = y[:, 1:]
logits = t_gpt2_forward(layers_params, y_in, mask, None, True, p_gen_aux) 

def print_res_shapes(res):
    print(len(res))
    for it in res:
        print(f'-', len(it))
        for p in it:
            print(f'--', p.shape, p.reshape(-1)[-3:]) # TODO: modify to check different parts

for i, i_mask in enumerate(mask):
    mask[i] = torch.tril(i_mask)
    #mask[i] = torch.zeros_like(i_mask)
#print(mask)

res2 = t_avg_cross_entropy_loss_bkwd(y_out, logits)
print(res2.shape, res2)
#print_res_shapes(res2[0]) 

print(f'----XXX----')

res3 = t_avg_cross_entropy_loss_bkwd2(y_out, logits)
print(res3.shape, res3)
#print_res_shapes(res3[0]) 


env: CUDA_VISIBLE_DEVICES=""


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!